In [1]:
import os
import pandas as pd
import nest_asyncio
import asyncio
from dotenv import load_dotenv
from openai import AsyncOpenAI
from datetime import datetime
from tqdm.asyncio import tqdm_asyncio

nest_asyncio.apply()
load_dotenv()
client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

TOPICS = {
    "food": "음식에 관한 친근한 스몰토크",
    "weather": "날씨에 대한 자연스러운 대화",
    "hobby": "취미 관련 이야기를",
}

def build_conversation_prompt(topic_kr, n_turns=10):
    return f"""{topic_kr}를 주제로 자연스럽고 편안한 {n_turns}턴짜리 대화를 만들어줘.

조건:
- 질문 → 답변 → 다시 묻기 방식
- "A:", "B:" 로 시작하는 라인 형식
- 말투는 일상적이고 자연스럽게
- 중복된 표현 피하고 다양한 표현 사용
"""

async def generate_one(prompt):
    try:
        response = await client.chat.completions.create(
            model="gpt-4.1-nano",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.85,
            max_tokens=1024,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"❌ 오류: {e}")
        return None

async def generate_turns(topic_key, topic_kr, n_samples, batch_size=20):
    results = set()
    total_generated = 0
    prompt = build_conversation_prompt(topic_kr)

    pbar = tqdm_asyncio(total=n_samples, desc=f"[{topic_key}] 생성 중")

    while total_generated < n_samples:
        tasks = [generate_one(prompt) for _ in range(batch_size)]
        responses = await asyncio.gather(*tasks)

        for r in responses:
            if r and r not in results:
                results.add(r)
                total_generated += 1
                pbar.update(1)

            if total_generated >= n_samples:
                break

    pbar.close()
    return list(results)

async def main(total_per_topic=5000):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    for topic_key, topic_kr in TOPICS.items():
        print(f"\n🔹 '{topic_key}' 주제 생성 시작...")
        results = await generate_turns(topic_key, topic_kr, total_per_topic)
        df = pd.DataFrame({"topic": topic_key, "conversation": results})
        filename = f"{topic_key}_chats_{timestamp}.csv"
        df.to_csv(filename, index=False, encoding="utf-8-sig")
        print(f"✅ 저장 완료: {filename} (총 {len(df)}개)")

# Jupyter 실행 시
await main(total_per_topic=5000)



🔹 'food' 주제 생성 시작...


[food] 생성 중: 100%|██████████| 5000/5000 [49:38<00:00,  1.68it/s]  


✅ 저장 완료: food_chats_20250731_223246.csv (총 5000개)

🔹 'weather' 주제 생성 시작...


[weather] 생성 중: 100%|██████████| 5000/5000 [44:45<00:00,  1.86it/s]  


✅ 저장 완료: weather_chats_20250731_223246.csv (총 5000개)

🔹 'hobby' 주제 생성 시작...


[hobby] 생성 중: 100%|██████████| 5000/5000 [1:11:49<00:00,  1.16it/s]

✅ 저장 완료: hobby_chats_20250731_223246.csv (총 5000개)
